# Prediction

## importation des données

In [10]:
import pandas as pd
import numpy as np

In [11]:
hotels = pd.read_csv('features_hotels.csv', index_col=['hotel_id', 'city'])
pricing_requests = pd.read_csv("price_dataset1.csv",index_col =0)
pricing_requests = pricing_requests.join(hotels, on=['hotel_id', 'city'])
pricing_requests


,hotel_id,price,stock,city,date,language,mobile,avatar_id,group,brand,parking,pool,children_policy
0,969,73,56,amsterdam,44,dutch,0,111612,Accar Hotels,Ibas,1,0,0
1,760,154,196,amsterdam,44,dutch,0,111612,Yin Yang,Royal Lotus,0,0,0
2,570,93,112,amsterdam,44,dutch,0,111612,Independant,Independant,1,0,0
3,835,97,113,amsterdam,44,dutch,0,111612,Independant,Independant,0,0,0
4,680,131,55,amsterdam,44,dutch,0,111612,Independant,Independant,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,177,164,0,vilnius,0,greek,0,111612,Accar Hotels,Marcure,0,1,0
89,47,163,0,vilnius,0,greek,0,111612,Accar Hotels,Marcure,1,0,0
90,187,168,0,vilnius,0,greek,0,111612,Accar Hotels,Marcure,1,1,0
91,136,140,0,vilnius,0,greek,0,111612,Boss Western,Boss Western,0,0,0


In [12]:
# def split_dataset(dataset, test_ratio=0.30):
#   """Splits a panda dataframe in two."""
#   test_indices = np.random.rand(len(dataset)) < test_ratio
#   return dataset[~test_indices], dataset[test_indices]


In [13]:
# train_ds_pd, test_ds_pd = split_dataset(pricing_requests)

In [14]:
import tensorflow_decision_forests as tfdf

2022-12-12 14:44:16.710513: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-12 14:44:25.684648: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/insa/lib:$LD_LIBRARY_PATH
2022-12-12 14:44:25.685202: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/insa/lib:$LD_LIBRARY_PATH
2022-12-12 14:44:25.685214: W tensorflow/compile

In [15]:
label ="price"
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(pricing_requests, label=label, task=tfdf.keras.Task.REGRESSION)
# test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label, task=tfdf.keras.Task.REGRESSION)

2022-12-12 14:44:35.911207: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-12 14:44:42.228212: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9951 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:65:00.0, compute capability: 6.1


## Tuner potentiel

In [8]:
# # Configure the tuner.

# # Create a Random Search tuner with 50 trials.
# tuner = tfdf.tuner.RandomSearch(num_trials=50)

# # Define the search space.
# #
# # Adding more parameters generaly improve the quality of the model, but make
# # the tuning last longer.

# tuner.choice("min_examples", [2, 5, 7, 10])
# tuner.choice("categorical_algorithm", ["CART", "RANDOM"])

# # Some hyper-parameters are only valid for specific values of other
# # hyper-parameters. For example, the "max_depth" parameter is mostly useful when
# # "growing_strategy=LOCAL" while "max_num_nodes" is better suited when
# # "growing_strategy=BEST_FIRST_GLOBAL".

# local_search_space = tuner.choice("growing_strategy", ["LOCAL"])
# local_search_space.choice("max_depth", [3, 4, 5, 6, 8])

# # merge=True indicates that the parameter (here "growing_strategy") is already
# # defined, and that new values are added to it.
# global_search_space = tuner.choice("growing_strategy", ["BEST_FIRST_GLOBAL"], merge=True)
# global_search_space.choice("max_num_nodes", [16, 32, 64, 128, 256])

# tuner.choice("use_hessian_gain", [True, False])
# tuner.choice("shrinkage", [0.02, 0.05, 0.10, 0.15])
# tuner.choice("num_candidate_attributes_ratio", [0.2, 0.5, 0.9, 1.0])

# # Uncomment some (or all) of the following hyper-parameters to increase the
# # quality of the search. The number of trial should be increased accordingly.

# # tuner.choice("split_axis", ["AXIS_ALIGNED"])
# # oblique_space = tuner.choice("split_axis", ["SPARSE_OBLIQUE"], merge=True)
# # oblique_space.choice("sparse_oblique_normalization",
# #                      ["NONE", "STANDARD_DEVIATION", "MIN_MAX"])
# # oblique_space.choice("sparse_oblique_weights", ["BINARY", "CONTINUOUS"])
# # oblique_space.choice("sparse_oblique_num_projections_exponent", [1.0, 1.5])


## Compilation et prédiction

In [16]:
model = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION)
model.compile(metrics=["mse"])
model.fit(x=train_ds, verbose =0)


Use /tmp/tmph0y8n26k as temporary training directory
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
[INFO 2022-12-12T14:45:09.920108143+01:00 kernel.cc:1175] Loading model from path /tmp/tmph0y8n26k/model/ with prefix fd361f102b3041cd
[INFO 2022-12-12T14:45:20.677327891+01:00 abstract_model.cc:1306] Engine "RandomForestOptPred" built
[INFO 2022-12-12T14:45:20.677380898+01:00 kernel.cc:1021] Use fast generic engine


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [17]:
# evaluation = model.evaluate(test_ds ,return_dict=True)
# si on a split nos données

# print(evaluation)

In [18]:
test = pd.read_csv("test_set.csv",index_col=0)

test = test.join(hotels, on=['hotel_id', 'city'])

test = test.drop(["order_requests"],axis =1)

test_df = tfdf.keras.pd_dataframe_to_tf_dataset(test)

A =model.predict(test_df)

7/7 [==============================] - 1s 102ms/step


In [19]:
test_df

<PrefetchDataset element_spec={'city': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'date': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'language': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'mobile': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'avatar_id': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'hotel_id': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'stock': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'group': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'brand': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'parking': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'pool': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'children_policy': TensorSpec(shape=(None,), dtype=tf.int64, name=None)}>

In [20]:
test

,city,date,language,mobile,avatar_id,hotel_id,stock,group,brand,parking,pool,children_policy
index,,,,,,,,,,,,
0,vilnius,21,romanian,0,1,161,46,Boss Western,J.Halliday Inn,1,0,0
1,vilnius,21,romanian,0,1,187,32,Accar Hotels,Marcure,1,1,0
2,vilnius,21,romanian,0,1,279,12,Independant,Independant,1,0,0
3,vilnius,21,romanian,0,1,395,10,Accar Hotels,Ibas,0,0,0
4,vilnius,21,romanian,0,1,488,42,Accar Hotels,Safitel,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
6639,rome,5,irish,0,794,987,1,Accar Hotels,Ibas,1,0,0
6640,vienna,1,irish,1,794,26,1,Accar Hotels,Marcure,1,0,0
6641,vienna,1,irish,1,794,263,0,Boss Western,Boss Western,1,0,0


In [12]:
Submission = pd.DataFrame(A)

Submission.to_csv("submission1.csv",header = ["price"],index_label="index")

# Gradio

In [32]:
def DfFromInput(city,date,language,mobile,avatar_id,hotel_id,stock):
    d = {'city':[city] ,'date':[date] , 'language':[language] ,'mobile':[mobile] ,'avatar_id':[avatar_id] ,
         'hotel_id':[hotel_id],'stock':[stock]}
    df=pd.DataFrame(data=d)
    return df

def AddData(city,date,language,mobile,avatar_id,hotel_id,stock):
    df = DfFromInput(city,date,language,mobile,avatar_id,hotel_id,stock)
    hotels = pd.read_csv('features_hotels.csv', index_col=['hotel_id', 'city'])
    df = df.join(hotels, on=['hotel_id', 'city'])
    return df

def Price(city,date,language,mobile,avatar_id,hotel_id,stock):
    df =AddData(city,date,language,mobile,avatar_id,hotel_id,stock)
    df =tfdf.keras.pd_dataframe_to_tf_dataset(df)
    price =model.predict(df,verbose =0)
    price=price[0,0]
    return price

TestDF = AddData("vilnius",21,"romanian",0,1,161,46)
price = Price("vilnius",21,"romanian",0,1,161,46)
price

114.11332

In [21]:
test

,city,date,language,mobile,avatar_id,hotel_id,stock,group,brand,parking,pool,children_policy
index,,,,,,,,,,,,
0,vilnius,21,romanian,0,1,161,46,Boss Western,J.Halliday Inn,1,0,0
1,vilnius,21,romanian,0,1,187,32,Accar Hotels,Marcure,1,1,0
2,vilnius,21,romanian,0,1,279,12,Independant,Independant,1,0,0
3,vilnius,21,romanian,0,1,395,10,Accar Hotels,Ibas,0,0,0
4,vilnius,21,romanian,0,1,488,42,Accar Hotels,Safitel,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
6639,rome,5,irish,0,794,987,1,Accar Hotels,Ibas,1,0,0
6640,vienna,1,irish,1,794,26,1,Accar Hotels,Marcure,1,0,0
6641,vienna,1,irish,1,794,263,0,Boss Western,Boss Western,1,0,0


In [9]:
testGrad = pd.read_csv("price_dataset1.csv",index_col =0)
testGrad

,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,969,73,56,amsterdam,44,dutch,0,111612
1,760,154,196,amsterdam,44,dutch,0,111612
2,570,93,112,amsterdam,44,dutch,0,111612
3,835,97,113,amsterdam,44,dutch,0,111612
4,680,131,55,amsterdam,44,dutch,0,111612
...,...,...,...,...,...,...,...,...
88,177,164,0,vilnius,0,greek,0,111612
89,47,163,0,vilnius,0,greek,0,111612
90,187,168,0,vilnius,0,greek,0,111612
91,136,140,0,vilnius,0,greek,0,111612


In [1]:
pip install gradio

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 34.1 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.1/213.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 31.3 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.9/144.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 35.7 MB/s eta 0:00

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 21.7 MB/s eta 0:00:00m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 4.4 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4693 sha256=3af7bfbc4678ffe6f9a96b7f4cf2fd8b79df133714a9d0a5c726e25b573d2885
  Stored in directory: /home/brosset/.cache/pip/wheels/91/e2/96/f676aa08bfd789328c6576cd0f1fde4a3d686703bb0c247697
  Created wheel for python-multipart: filename=python_multipart-0.0.5-py3-none-any.whl size=31671 sha256=0c63bf3fbd8d56caf15577f2878215fd1be998def0f969bb5a2a27539f170add
  Stored in directory: /home/brosset/.cache/pip/wheels/fe/04/d1/a10661cc45f03c3cecda50deb2d2c22f57b4e84a75b2a5987e
Successfully built ffmpy python-multipart
  Consider adding this directory to PATH or, if you prefer to suppress this w

In [2]:
import gradio as gr

In [ ]:
(city,date,language,mobile,avatar_id,hotel_id,stock)

In [35]:
test = gr.Interface(fn = Price,
                    inputs= [
                        gr.Radio(["amsterdam","copenhagen","madrid","paris","rome",
                                          "sofia","valletta","vienna","vilnius"]),
                        gr.Slider(0,44,step =1,value =0),
                        gr.Radio(["austrian","belgian","bulgarian","croatian","cypriot",
                                       "czech","danish","dutch","estonian","finnish","french",
                                       "german","greek","hungarian","irish","italian","latvian",
                                       "lithuanian","luxembourgish","maltese","polish","portuguese",
                                       "romanian","slovakian","slovene","spanish","swedish"]),
                        gr.Slider(0,1,step=1,value=0),
                        gr.Number(value=111612),
                        gr.Number(),
                        gr.Number()
                        
                    ],
                    
                    outputs="number")
test.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
